In [2]:
import os
import getpass

if not os.environ.get("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key: ")

INDEX_NAME = "aion2-guide-rag"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

vector_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("Pinecone 벡터 저장소 초기화 완료")

d:\py_work\rag_test13\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pinecone 벡터 저장소 초기화 완료


In [4]:
test_query = "이 게임의 수호성 스킬은 어떤게 있어?"

docs = retriever.invoke(test_query)
print(f"검색 결과: {len(docs)}개의 문서")
print("-"*30)
for i, doc in enumerate(docs, 1):
    print(f"[문서 {i+1}] {doc.page_content[:100]}...")

d:\py_work\rag_test13\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


검색 결과: 3개의 문서
------------------------------
[문서 2] 문서 제목: 수호성 스킬
문서 요약: 수호성 클래스 스킬 정보입니다...
[문서 3] 문서 제목: 궁성 스킬
문서 요약: 궁성 클래스 스킬 정보입니다

내용:
마스터리 스킬
아이콘
명칭
설명
비고
저격 - 연사 - 나선 화살
적들에게 피해를 주고, 정신력을 회복합니...
[문서 4] 스티그마 스킬
스티그마 시스템은 고급 스킬을 추가로 사용할 수 있는 시스템입니다.
스티그마 스킬은 일반 스킬 포인트가 아닌 ‘스티그마 샤드’를 통해 레벨을 상승시킬 수 있습니다.
...


RAG 체인 생성 완료!


In [9]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_cohere import CohereRerank
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever


embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)


retriever = vector_store.as_retriever(search_kwargs={"k": 20})
# test_query = "이 게임의 궁성 스킬은 어떤게 있어?"

# docs = retriever.invoke(test_query)



compressor = CohereRerank(
    model="rerank-multilingual-v3.0", # 한국어 지원이 훌륭한 모델
    top_n=5 
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

query = "수호성 스킬 알려줘"

final_docs = compression_retriever.invoke(query)

for doc in final_docs:
    print(f"점수: {doc.metadata['relevance_score']} | 내용: {doc.page_content[:1000]}...")

점수: 0.994615 | 내용: 수호성
수호성
적의 공격을 차단하고 아군을 보호하는
전형적인 탱커 클래스
입니다.
수호성은
장검과 방패
를 활용해 높은 방어력을 자랑하며,
자신뿐만 아니라 아군을 보호하는 다양한 방어 스킬을 보유하고 있습니다.
파티 플레이 시
최전선에서 전투의 중심을 지키는 수호자
의 역할을 수행합니다.
착용 무기:
장검
[수호성 스킬 정보 보러가기]...
점수: 0.9698426 | 내용: 문서 제목: 수호성
문서 요약: 방패로 아군을 보호하는 철벽의 수호자, 수호성 클래스 소개입니다.

내용:
수호성
수호성
적의 공격을 차단하고 아군을 보호하는
전형적인 탱커 클래스
입니다.
수호성은
장검과 방패
를 활용해 높은 방어력을 자랑하며,
자신뿐만 아니라 아군을 보호하는 다양한 방어 스킬을 보유하고 있습니다.
파티 플레이 시
최전선에서 전투의 중심을 지키는 수호자
의 역할을 수행합니다.
착용 무기:
장검
[수호성 스킬 정보 보러가기]...
점수: 0.5436811 | 내용: 문서 제목: 수호성 스킬
문서 요약: 수호성 클래스 스킬 정보입니다...
점수: 0.46663293 | 내용: 문서 제목: 호법성 스킬
문서 요약: 호법성 스킬 정보입니다.

내용:
마스터리 스킬
아이콘
명칭
설명
비고
격파쇄 - 공명쇄 - 벽력쇄
적들에게 피해를 주고, 정신력을 회복합니다.
논타겟/이동 가능
백열격 - 파열격
적들에게 피해를 줍니다.
그로기 게이지 피해
논타겟/이동 가능
돌진 격파
대상에게 돌진하여 적들에게 피해를 줍니다.
그로기 게이지 피해
-
타격쇄
대상에게 피해를 주고, 일정 확률로 기절 상태로 만듭니다.
대상이 NPC일 경우 100%의 확률로 기절이 적중됩니다.
그로기 게이지 피해
-
암격쇄
기절, 넘어짐, 공중 속박 상태의 대상에게 피해를 줍니다.
-
질풍 난타
그로기 상태인 대상에게 피해를 줍니다.
-
열파격
막기 성공 시, 대상을 중심으로 적들에게 피해를 주고, 일정 확률로 봉인 상태로 만듭니다.
대상이 NPC일 경우 100%의 확률로 봉인이 적중

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
당신은 게임 초보자를 도와주는 친절한 '게임 가이드 AI'입니다.
아래의 [게임 가이드북 내용]을 참고하여 질문에 대해 정확하고 이해하기 쉽게 답변해주세요.
가이드북에 없는 내용은 지어내지 말고 솔직하게 모른다고 답변하세요.

[게임 가이드북 내용]
{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print("RAG 체인 생성 완료!")

RAG 체인 생성 완료!


In [1]:
query = "수호성 스킬에 대한 가이드를 알려줘"

print(f"질문: {query}")
print("답변 생성중...")
print("-"*50)

response = rag_chain.invoke(query)

print(response)

질문: 수호성 스킬에 대한 가이드를 알려줘
답변 생성중...
--------------------------------------------------


NameError: name 'rag_chain' is not defined

In [9]:
query = " 던전 초월 콘텐츠를 알려줘"

print(f"질문: {query}")
print("답변 생성중...")
print("-"*50)

response = rag_chain.invoke(query)

print(response)

질문:  던전 초월 콘텐츠를 알려줘
답변 생성중...
--------------------------------------------------
초월 던전은 일반 원정보다 높은 난이도로 제공되는 던전 콘텐츠입니다. 이 콘텐츠는 플레이어가 랭킹을 경쟁하며 즐길 수 있는 요소가 있으며, 보상으로 ‘돌파석 조각’과 ‘아르카나’를 획득할 수 있습니다. 초월 던전에 입장하기 위해서는 도전 횟수가 필요하며, 초월 보스를 클리어한 후 보상을 받을 때 도전 횟수가 소모됩니다. 만약 던전에서 클리어 후 보상을 받지 않으면 퇴장 시 도전 횟수는 소모되지 않습니다. 초월 도전 횟수는 매일 총 2회가 충전됩니다. 초월 단계는 1단계부터 10단계까지 있으며, 단계가 높아질수록 난이도와 보상이 상승합니다.
